<a href="https://colab.research.google.com/github/zeiosis/ffnet-summary-prediction/blob/main/dangerzone/colab/stats_testing_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
rawtext_filename = 'manualtest1.txt'

# Library imports
Always run.

In [ ]:
!pip install -U transformers==4.15.0
!pip install -U sentence-transformers==2.2.0
!pip install -U transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.4 MB 7.4 MB/s 
     |████████████████████████████████| 596 kB 57.6 MB/s 
     |████████████████████████████████| 3.3 MB 33.2 MB/s 
     |████████████████████████████████| 86 kB 4.7 MB/s 
     |████████████████████████████████| 880 kB 51.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=718f808727ef77c7d1e4b3023534b09c763f0a53cb62dea54705c3bda17c3394
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 3.5 MB/s 
     |███████

In [ ]:
import transformers
from transformers import pipeline
import pandas as pd

In [ ]:
import torch

## Raw text to csv function for ffnet

In [ ]:
droplist = ['Crime',
 'Fantasy',
 'General',
 'Horror',
 'Mystery',
 'Parody',
 'Poetry',
 'Sci_Fi',
 'Spiritual',
 'Supernatural',
 'Suspense',
 'Tragedy',
 'Western']

def webtext2dict(text):
    from re import findall
    slistlite = findall(r"iews\n.*\nR", text)
    slist = []
    for x in slistlite:
        slist.append(x[5:len(x)-2])
        
    glistlite = findall(r"h - .*? - C|h - Ch", text)
    glist = []
    for x in glistlite:
        glist.append(x[4:len(x)-4])
    
    sumgenr_dict = {}
    for x in range(len(slist)):
        sumgenr_dict[slist[x]] = glist[x]
        
    keyremoveset = set()
    for key in sumgenr_dict:
        if len(sumgenr_dict[key]) > 30:
            keyremoveset.add(key)
    
    for x in keyremoveset:
        sumgenr_dict.pop(x, None)

    return sumgenr_dict

def dictclean(inputdict):
    #1st pass for Hurt/Comfort > Hurt_Comfort, s-f>s_f naming inconsistencies b/c vars dont support dashes
    #2nd pass to get rid of other slashes (k/j > [k, j])

    from re import sub
    
    hc_repl_list = []
    
    for key in inputdict:
        if 'Hurt/Comfort' in inputdict[key]:
            hc_repl_list.append(key)
            
    for x in hc_repl_list:
        inputdict[x] = sub('Hurt/Comfort', 'Hurt_Comfort', inputdict[x])
        
        
    sf_repl_list = []
    
    for key in inputdict:
        if 'Sci-Fi' in inputdict[key]:
            sf_repl_list.append(key)
            
    for x in sf_repl_list:
        inputdict[x] = sub('Sci-Fi', 'Sci_Fi', inputdict[x])
    
    #1st pass done
    
    for key in inputdict:
        inputdict[key] = inputdict[key].split('/')
        
    #2nd pass done 
    
def ffdict_to_df(dfdict):
    import pandas as pd
    df = pd.DataFrame()
    df = df.assign(Summary=0, Adventure=0, Angst=0, Crime=0, Drama=0, Family=0,\
                   Fantasy=0, Friendship=0, General=0, Horror=0, Humor=0,\
                   Hurt_Comfort=0, Mystery=0, Parody=0, Poetry=0, Romance=0,\
                   Sci_Fi=0, Spiritual=0, Supernatural=0, Suspense=0, Tragedy=0, Western=0) #works for ffnet.
    for x in dfdict:
        r2 = {}
        r2['Summary'] = x
        for values in dfdict[x]:
            if values != '':
                r2[values] = 1
        df = df.append(r2, ignore_index=True)
        
    df = df.fillna(0)
    return df

def ffcoll_to_df(filename, drop_rare=False):
    #basically just go on ffnet and control a control c everything into a txt file.
    #This works with multiple pages in one file as well.
    f = open(filename, "r")
    rawtext1 = f.read()
    f.close 
    
    r1 = webtext2dict(rawtext1)
    dictclean(r1)
    df = ffdict_to_df(r1)
    
    if drop_rare == False:
        return df
    else:
        df = df.drop(labels=droplist, axis=1)
        return df

# Manual classification
Annoying but can help with debugging. 
Is not meant to work with predict_labels function; redundant.

In [ ]:
classifier = pipeline(model='zdreiosis/ff_analysis_3', return_all_scores=True) #return_all_scores is important

NameError: ignored

In [ ]:
labelpreds = classifier(['Two knights go on an adventure.'])
print(labelpreds)

In [ ]:
ksorted = [sorted(labelpreds[y], key=lambda x: x['score'], reverse=True) for y in range(len(labelpreds))]
ksorted

In [ ]:
for x in labelpreds[0]:
  print(x['label'] + ':', x['score'])

# Automatic classification
Easy mode. Set up list of summmaries by uploading from local and using pandas read_csv and extracting, or alternatively just make your own.

In [ ]:
def predict_labels(summary_list, revision, model='zdreiosis/ff_analysis_2'): #defaults to 'zdreiosis/ff_analysis_2', might change in future
  #there should not be any reason to call this function more than once. If there is, rewrite, b/c it is v. slow
  try:
    classifier = pipeline(model=model, revision=revision, return_all_scores=True, device=cuda_id)
  except:
    classifier = pipeline(model=model, revision=revision, return_all_scores=True, device=-1)
  return classifier(summary_list)

def preds_to_csv(predicted_labels, csvname):
  ksorted = [sorted(predicted_labels[y], key=lambda x: x['score']) for y in range(len(predicted_labels))]

  indexed_label_count = len(ksorted[0]) - 1
  reducedlist = [[x[indexed_label_count]['label'], x[indexed_label_count-1]['label']] for x in ksorted]

  strredlist = [','.join(x) for x in reducedlist]
  todf_predlist = [[summary_list[x], strredlist[x]] for x in range(len(strredlist))]

  preds_df = pd.DataFrame(todf_predlist, columns =['summary', 'tags'])
  preds_df = pd.concat([preds_df.drop('tags', axis=1), preds_df['tags'].str.get_dummies(',')], axis=1) #very useful code, ty stackoverflow

  preds_df.to_csv(csvname, index=False) 

def summaries_to_pred_csv(summary_list, model, revision, csvname):
  preds_to_csv(predict_labels(summary_list=summary_list, model=model, revision=revision), csvname)

def ffdict_to_df_test(dfdict_filename):
    import ast
    df = pd.DataFrame()
    df = df.assign(Summary=0, Adventure=0, Angst=0, Crime=0, Drama=0, Family=0,\
                   Fantasy=0, Friendship=0, General=0, Horror=0, Humor=0,\
                   Hurt_Comfort=0, Mystery=0, Parody=0, Poetry=0, Romance=0,\
                   Sci_Fi=0, Spiritual=0, Supernatural=0, Suspense=0, Tragedy=0, Western=0) #works for ffnet.
    f = open(dfdict_filename, "r")
    dfdict = f.read()
    dfdict = ast.literal_eval(dfdict)
    f.close 
    for x in dfdict:
        r2 = {}
        r2['Summary'] = x
        for values in dfdict[x]:
            if values != '':
                r2[values] = 1
        df = df.append(r2, ignore_index=True)
        
    df = df.fillna(0)
    return df


In [ ]:
df_summary = ffcoll_to_df(rawtext_filename, drop_rare=True) #converts raw txt to df

In [ ]:
df_summary = ffdict_to_df_test(rawtext_filename)

In [ ]:
droplist = ['Crime',
 'Fantasy',
 'General',
 'Horror',
 'Mystery',
 'Parody',
 'Poetry',
 'Sci_Fi',
 'Spiritual',
 'Supernatural',
 'Suspense',
 'Tragedy',
 'Western']
df_summary = df_summary.drop(labels=droplist, axis=1)

In [ ]:
df_summary

,Summary,Adventure,Angst,Drama,Family,Friendship,Humor,Hurt_Comfort,Romance
0,(One-Shot) Rin notices Sesshomaru's softer sid...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,A one-shot about how Inuyasha sees Kagome look...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,Kagome comes home to study for an exam but onl...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,Do you ever wonder what it would be like if Se...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,Beyond the cypress trees and the rivers windin...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
101,"A fun and interesting AU, where all the DBZ ch...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
102,The Cell games is in ten days. Gohan seems to ...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
103,Saiyaman and Videl enjoy a short afternoon wal...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
104,A new type of Fighting Tournament has been cre...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
summary_list = list(df_summary['Summary']) #excises summaries (first col)

In [ ]:
df_summary = pd.read_csv('ffdump7.csv')
summary_list = list(df_summary['Summary'])

In [ ]:
df_summary.tail()

,Unnamed: 0,Summary,Adventure,Angst,Drama,Family,Friendship,Humor,Hurt_Comfort,Romance
217,217,Forced to be sociable by his so called friends...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
218,218,Sequel to Tears of Rain: [SADLY DISCONTINUED],0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
219,219,"Natsu and Erza had never met in their lives, b...",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
220,220,Everything has its own consequences. Lucy's de...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
221,221,(Follow-up to my previous MidLu Halloween stor...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [ ]:
def preds_to_csv_test(predicted_labels, csvname):
  ksorted = [sorted(predicted_labels[y], key=lambda x: x['score']) for y in range(len(predicted_labels))]

  indexed_label_count = len(ksorted[0]) - 1
  reducedlist = [[x[indexed_label_count]['label'], x[indexed_label_count-1]['label']] for x in ksorted]

  strredlist = [','.join(x) for x in reducedlist]
  todf_predlist = [[summary_list[x], strredlist[x]] for x in range(len(strredlist))]

  preds_df = pd.DataFrame(todf_predlist, columns =['summary', 'tags'])
  preds_df = pd.concat([preds_df.drop('tags', axis=1), preds_df['tags'].str.get_dummies(',')], axis=1) #very useful code, ty stackoverflow

  return preds_df
  #preds_df.to_csv(csvname, index=False) 

In [ ]:
summaries_to_pred_csv(summary_list, model='zdreiosis/ff_analysis_5', revision='1454370', csvname='PREDS_zdr5_mantest_1.csv')

Downloading:   0%|          | 0.00/2.78k [00:00<?, ?B/s]

In [ ]:
classifier = pipeline(model='zdreiosis/ff_analysis_5', revision='1454370', return_all_scores=True, device=cuda_id) #commit_id aaa
csl = classifier(summary_list)
xdf = preds_to_csv_test(classifier(summary_list), 'idk.csv')

Downloading:   0%|          | 0.00/2.78k [00:00<?, ?B/s]

In [ ]:
xdf.tail()

,summary,Angst,Drama,Family,Friendship,Humor,Hurt_Comfort,Romance
217,Forced to be sociable by his so called friends...,0,0,0,0,0,1,1
218,Sequel to Tears of Rain: [SADLY DISCONTINUED],0,0,0,1,0,0,1
219,"Natsu and Erza had never met in their lives, b...",0,0,0,0,0,1,1
220,Everything has its own consequences. Lucy's de...,0,0,0,1,0,0,1
221,(Follow-up to my previous MidLu Halloween stor...,0,0,0,1,0,0,1


In [ ]:
ksorted = [sorted(csl[y], key=lambda x: x['score']) for y in range(len(csl))]
indexed_label_count = len(ksorted[0]) - 1
reducedlist = [[x[indexed_label_count]['label'], x[indexed_label_count-1]['label']] for x in ksorted]
strredlist = [','.join(x) for x in reducedlist]
todf_predlist = [[summary_list[x], strredlist[x]] for x in range(len(strredlist))]
preds_df = pd.DataFrame(todf_predlist, columns =['summary', 'tags'])
preds_df = pd.concat([preds_df.drop('tags', axis=1), preds_df['tags'].str.get_dummies(',')], axis=1)
preds_df
canon_cols = ['Adventure', 'Angst', 'Drama', 'Family', 'Friendship', 'Humor', 'Hurt_Comfort', 'Romance']


,summary,Angst,Drama,Family,Friendship,Humor,Hurt_Comfort,Romance
0,Ignia loses his little brother while Natsu is ...,0,0,0,0,0,1,1
1,This is my first fanfiction. The story of Nats...,0,0,0,1,0,0,1
2,In which Natsu fails to ask a very important q...,0,0,0,0,1,0,1
3,Gruvia drabbles and one shots!,0,0,0,0,1,0,1
4,The blood was so unbelievably rare. Who would ...,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...
217,Forced to be sociable by his so called friends...,0,0,0,0,0,1,1
218,Sequel to Tears of Rain: [SADLY DISCONTINUED],0,0,0,1,0,0,1
219,"Natsu and Erza had never met in their lives, b...",0,0,0,0,0,1,1
220,Everything has its own consequences. Lucy's de...,0,0,0,1,0,0,1


#Some more testing

In [ ]:
modelnames = ['facebook/bart-large-mnli', 'typeform/distilbert-base-uncased-mnli', 'valhalla/distilbart-mnli-12-1']
df_summary = ffcoll_to_df(rawtext_filename, drop_rare=True)
summary_list = list(df_summary['Summary'])


def model_stats(modelname, test_list, csvname):
  classifier = pipeline(model=modelname, device=cuda_id)
  fsumlist = test_list

  fromsum = classifier(
    fsumlist,
    candidate_labels=['Adventure', 'Angst', 'Drama', 'Family', 'Friendship',
       'Humor', 'Hurt_Comfort', 'Romance']
  )

  strredlist = [str(fromsum[x]['labels'][0] + ',' + fromsum[x]['labels'][1]) for x in range(len(fromsum))]
  todf_predlist = [[summary_list[x], strredlist[x]] for x in range(len(strredlist))]

  preds_df = pd.DataFrame(todf_predlist, columns =['summary', 'tags'])
  preds_df = pd.concat([preds_df.drop('tags', axis=1), preds_df['tags'].str.get_dummies(',')], axis=1) #very useful code, ty stackoverflow
  preds_df.to_csv(str(str(csvname) + '.csv'), index=False) 

In [ ]:
model_stats('valhalla/distilbart-mnli-12-1', summary_list, '15_jun_2_valh_preds')
model_stats('typeform/distilbert-base-uncased-mnli', summary_list, '15_jun_2_typf_preds')
model_stats('facebook/bart-large-mnli', summary_list, '15_jun_2_fb_preds')

Downloading:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/849M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]